In [1]:
"""
Use the MIWAE and not-MIWAE on UCI data
"""
import numpy as np
import pandas as pd
import os
import sys
sys.path.append(os.getcwd())
from MIWAE import MIWAE
from notMIWAE import notMIWAE
import trainer
import utils
from missing_util import introduce_mising, preprocessing,introduce_mising_advanced
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:

# ---- data settings
name = '/tmp/uci/task01/best'
n_hidden = 128
n_samples = 20
max_iter = 100000
batch_size = 16
L = 10000

# ---- choose the missing model
# mprocess = 'linear'
# mprocess = 'selfmasking'
mprocess = 'selfmasking_known'

# ---- number of runs
runs = 1
RMSE_miwae = []
RMSE_notmiwae = []
RMSE_notmiwae_selfmasking = []
RMSE_notmiwae_linear = []
RMSE_mean = []
RMSE_mice = []
RMSE_RF = []




C:\Users\Epochoid\AppData\Local\Temp\ipykernel_22636\1729541190.py:45: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  S = np.array(~np.isnan(Xnan), dtype=np.float)


In [56]:
def introduce_mising(X):
    N, D = X.shape
    Xnan = X.copy()

    # using mean as indicator
    # ---- MNAR in D/2 dimensions

    q3 = np.quantile(Xnan[:, :int(D / 2)],0.75, axis=0)
    q1 = np.quantile(Xnan[:, :int(D / 2)],0.25, axis=0)
    ix_q3 = Xnan[:, :int(D / 2)] > q3
    ix_q1 = Xnan[:, :int(D / 2)] < q1
    #ix_larger_than_mean = Xnan[:, :int(D / 2)] > mean
    Xnan[:, :int(D / 2)][ix_q3] = np.nan
    Xnan[:, :int(D / 2)][ix_q1] = np.nan

    Xz = Xnan.copy()
    Xz[np.isnan(Xnan)] = 0

    return Xnan, Xz

In [57]:
data_shape, Xtrain,Xval_org, dl = preprocessing("url1")
Xnan, Xz = introduce_mising(Xtrain)

In [58]:
np.isnan(Xnan).sum()

1372

In [41]:
# full = 5484



(1371, 4)

In [40]:
np.count_nonzero(~np.isnan(Xnan))

4112

In [22]:
for _ in range(runs):

    


    data_shape, Xtrain,Xval_org, dl = preprocessing("url1")

    # ---- introduce missing process
    Xnan, Xz = introduce_mising(Xtrain)
    #Xnan, Xz =introduce_mising_advanced(Xtrain, 0.8 , "MNAR")
    # mask
    S = np.array(~np.isnan(Xnan), dtype=np.float)
    Xval, Xvalz = introduce_mising(Xval_org)

C:\Users\Epochoid\AppData\Local\Temp\ipykernel_22636\541635747.py:12: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  S = np.array(~np.isnan(Xnan), dtype=np.float)


,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.284354
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.178794
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.696601
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.768036
